### show files system hierachy

In [1]:
from pathlib import Path
import os

class DisplayablePath(object):
    display_filename_prefix_middle = '├──'
    display_filename_prefix_last = '└──'
    display_parent_prefix_middle = '    '
    display_parent_prefix_last = '│   '

    def __init__(self, path, parent_path, is_last):
        self.path = Path(str(path))
        self.parent = parent_path
        self.is_last = is_last
        if self.parent:
            self.depth = self.parent.depth + 1
        else:
            self.depth = 0

    @property
    def displayname(self):
        if self.path.is_dir():
            return self.path.name + '/'
        return self.path.name

    @classmethod
    def make_tree(cls, root, parent=None, is_last=False, criteria=None):
        root = Path(str(root))
        criteria = criteria or cls._default_criteria

        displayable_root = cls(root, parent, is_last)
        yield displayable_root

        children = sorted(list(path
                               for path in root.iterdir()
                               if criteria(path)),
                          key=lambda s: str(s).lower())
        count = 1
        for path in children:
            is_last = count == len(children)
            if path.is_dir():
                yield from cls.make_tree(path,
                                         parent=displayable_root,
                                         is_last=is_last,
                                         criteria=criteria)
            else:
                yield cls(path, displayable_root, is_last)
            count += 1

    @classmethod
    def _default_criteria(cls, path):
        return True

    @property
    def displayname(self):
        if self.path.is_dir():
            return self.path.name + '/'
        return self.path.name

    def displayable(self):
        if self.parent is None:
            return self.displayname

        _filename_prefix = (self.display_filename_prefix_last
                            if self.is_last
                            else self.display_filename_prefix_middle)

        parts = ['{!s} {!s}'.format(_filename_prefix,
                                    self.displayname)]

        parent = self.parent
        while parent and parent.parent is not None:
            parts.append(self.display_parent_prefix_middle
                         if parent.is_last
                         else self.display_parent_prefix_last)
            parent = parent.parent

        return ''.join(reversed(parts))

In [15]:
# With a criteria (skip hidden files)
def is_not_hidden(path):
    if path.name.startswith("."): 
        return False
    else:
        print(path)
        return True
    


In [16]:
root_dir = os.path.expanduser('../')
paths = DisplayablePath.make_tree(
    Path(root_dir),
    criteria=is_not_hidden
)
for path in paths:
    print(path.displayable())

paths = DisplayablePath.make_tree(Path(root_dir), criteria=is_not_hidden)
for path in paths:
    print(path.displayable())

../
../notebooks
../requirements.txt
../src.egg-info
../setup.py
../data
../src
../README.md
├── data/
../data/raw
../data/processed
│   ├── processed/
../data/processed/lyon
../data/processed/SIREN
│   │   ├── lyon/
../data/processed/lyon/2023
../data/processed/lyon/BDTOPO
│   │   │   ├── 2023/
../data/processed/lyon/2023/ZoneEtude
../data/processed/lyon/2023/SIREN
│   │   │   │   ├── SIREN/
../data/processed/lyon/2023/SIREN/GeoSiren_lyon_25km.parquet
│   │   │   │   │   └── GeoSiren_lyon_25km.parquet
│   │   │   │   └── ZoneEtude/
../data/processed/lyon/2023/ZoneEtude/ze_lyon_25km.gpkg
│   │   │   │       └── ze_lyon_25km.gpkg
│   │   │   └── BDTOPO/
../data/processed/lyon/BDTOPO/2008
../data/processed/lyon/BDTOPO/2023
../data/processed/lyon/BDTOPO/2013
│   │   │       ├── 2008/
../data/processed/lyon/BDTOPO/2008/communes_2008.shp
../data/processed/lyon/BDTOPO/2008/communes_2008.cpg
../data/processed/lyon/BDTOPO/2008/bati_indus_lyon_2008.cpg
../data/processed/lyon/BDTOPO/2008/bati_in